In [79]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

In [81]:
df = pd.read_csv('train.csv')

In [83]:
df.shape

(7613, 5)

In [85]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [87]:
print((df.target == 1).sum())#disaster
print((df.target == 0).sum())#no disaster

3271
4342


In [89]:
import re
import string
def remove_URL(text):
    url = re.compile(r"https?://\s+|www\.\s+")
    return url.sub(r" ",text)

def remove_punct(text):
    translator = str.maketrans("","",string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [91]:
pattern = re.compile(r"https?://(\s+|www)\.\s+")
for t in df.text:   
    matches = pattern.findall(t)
    for match in matches:
        print(t)
        print(match)
        print(pattern.sub(r'',t))
        if len(matches) > 0:
            break

In [93]:
df["text"] = df.text.map(remove_URL)
df["text"] = df.text.map(remove_punct)

In [95]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = set (stopwords.words("english"))
def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " " . join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [97]:
stop

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [99]:
df["text"] = df.text.map(remove_stopwords)

In [101]:
df.text

0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       13000 people receive wildfires evacuation orde...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    ariaahrary thetawniest control wild fires cali...
7610    m194 0104 utc5km volcano hawaii httptcozdtoyd8ebj
7611    police investigating ebike collided car little...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

In [113]:
!pip install collection

In [103]:
from collections import Counter

def count_words(texts):
    count = Counter()
    for text in texts:
        for word in text.split():
            count[word] += 1
    return count

Counter = count_words(df.text)


counter_length = len(Counter)
print("Number of unique words:", counter_length)


Number of unique words: 22564


In [105]:
from collections import Counter

def count_words(texts):
    count = Counter()
    for text in texts:
        for word in text.split():
            count[word] += 1
    return count


Counter_object = count_words(df['text'])


num_unique_words = len(Counter_object)
print("Number of unique words:", num_unique_words)


most_common_words = Counter_object.most_common(5)
print("5 most common words:", most_common_words)


Number of unique words: 22564
5 most common words: [('like', 345), ('im', 299), ('amp', 298), ('fire', 250), ('get', 229)]


In [107]:
train_size = int(df.shape[0] * 0.8)
train_df = df[:train_size]
val_df = df[train_size:]
train_sentences = train_df.text.to_numpy()
train_labels = train_df.target.to_numpy()
val_sentences = val_df.text.to_numpy()
val_labels = val_df.target.to_numpy()



In [109]:
train_sentences_length = len(train_sentences)
val_sentences_length = len(val_sentences)

print("Train Sentences Length:", train_sentences_length)
print("Validation Sentences Length:", val_sentences_length)


Train Sentences Length: 6090
Validation Sentences Length: 1523


In [111]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences)
tokenizer.fit_on_texts(val_sentences)

In [113]:
word_index = tokenizer.word_index

In [115]:
word_index

{'like': 1,
 'im': 2,
 'amp': 3,
 'fire': 4,
 'get': 5,
 'new': 6,
 'via': 7,
 'people': 8,
 'one': 9,
 'news': 10,
 'dont': 11,
 'video': 12,
 'us': 13,
 '2': 14,
 'emergency': 15,
 'disaster': 16,
 'police': 17,
 'would': 18,
 'still': 19,
 'body': 20,
 'burning': 21,
 'crash': 22,
 'back': 23,
 'california': 24,
 'storm': 25,
 'suicide': 26,
 'got': 27,
 'time': 28,
 'know': 29,
 'man': 30,
 'buildings': 31,
 'day': 32,
 'rt': 33,
 'first': 34,
 'see': 35,
 'going': 36,
 'world': 37,
 'bomb': 38,
 'cant': 39,
 'nuclear': 40,
 'love': 41,
 'fires': 42,
 'attack': 43,
 'youtube': 44,
 'two': 45,
 '3': 46,
 'dead': 47,
 'killed': 48,
 'go': 49,
 'train': 50,
 'full': 51,
 'car': 52,
 'war': 53,
 'may': 54,
 'families': 55,
 'life': 56,
 'accident': 57,
 'today': 58,
 'good': 59,
 'hiroshima': 60,
 'think': 61,
 'say': 62,
 'watch': 63,
 'many': 64,
 'last': 65,
 'u': 66,
 'could': 67,
 'want': 68,
 'years': 69,
 'way': 70,
 'home': 71,
 'make': 72,
 'collapse': 73,
 'work': 74,
 'look'

In [117]:


# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)




In [119]:
print(train_sentences[10:15])
print(val_sequences[10:15])

['three people died heat wave far'
 'haha south tampa getting flooded hah wait second live south tampa gonna gonna fvck flooding'
 'raining flooding florida tampabay tampa 18 19 days ive lost count'
 'flood bago myanmar arrived bago'
 'damage school bus 80 multi car crash breaking']
[[1268, 2631, 269, 188, 306, 1436, 998, 1652], [1034, 19541, 188, 693], [188, 693, 19542, 19543, 1587, 19544, 3209, 1866, 19545, 19546], [340, 188, 643, 1964, 71, 297, 1965, 1966, 232, 28], [169, 6475, 1829, 2419, 67, 19547, 19548, 2649, 149, 188]]


In [121]:
from tensorflow.keras.preprocessing.sequence import pad_sequences



# Convert text sentences to sequences of integers
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [123]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


max_length = 20
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")

# Print shapes
print("Shape of train_padded:", train_padded.shape)
print("Shape of val_padded:", val_padded.shape)
train_padded[10]

Shape of train_padded: (6090, 20)
Shape of val_padded: (1523, 20)


array([488,   8, 507, 184, 366, 508,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])

In [125]:
print(train_sentences[10])
print(train_sequences[10])
print(train_padded[10])

three people died heat wave far
[488, 8, 507, 184, 366, 508]
[488   8 507 184 366 508   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]


In [127]:
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [51]:
reverse_word_index

{1: 'like',
 2: 'im',
 3: 'amp',
 4: 'fire',
 5: 'get',
 6: 'new',
 7: 'via',
 8: 'people',
 9: 'one',
 10: 'news',
 11: 'dont',
 12: 'video',
 13: 'us',
 14: '2',
 15: 'emergency',
 16: 'disaster',
 17: 'police',
 18: 'would',
 19: 'still',
 20: 'body',
 21: 'burning',
 22: 'crash',
 23: 'back',
 24: 'california',
 25: 'storm',
 26: 'suicide',
 27: 'got',
 28: 'time',
 29: 'know',
 30: 'man',
 31: 'buildings',
 32: 'day',
 33: 'rt',
 34: 'first',
 35: 'see',
 36: 'going',
 37: 'world',
 38: 'bomb',
 39: 'cant',
 40: 'nuclear',
 41: 'love',
 42: 'fires',
 43: 'attack',
 44: 'youtube',
 45: 'two',
 46: '3',
 47: 'dead',
 48: 'killed',
 49: 'go',
 50: 'train',
 51: 'full',
 52: 'car',
 53: 'war',
 54: 'may',
 55: 'families',
 56: 'life',
 57: 'accident',
 58: 'today',
 59: 'good',
 60: 'hiroshima',
 61: 'think',
 62: 'say',
 63: 'watch',
 64: 'many',
 65: 'last',
 66: 'u',
 67: 'could',
 68: 'want',
 69: 'years',
 70: 'way',
 71: 'home',
 72: 'make',
 73: 'collapse',
 74: 'work',
 75: 'l

In [129]:
 def decode(sequence):
     return " ".join([reverse_word_index.get(idx,"?") for idx in sequence])

In [131]:
decoded_text = decode(train_sequences[10])
print(train_sequences[10])
print(decoded_text)


[488, 8, 507, 184, 366, 508]
three people died heat wave far


In [73]:
!pip install keras

In [71]:
pip install --upgrade tensorflow


In [138]:
inputs = keras.Input(shape=(max_length,))
x = layers.Embedding(input_dim=num_unique_words, output_dim=32)(inputs)
x = layers.LSTM(64, dropout=0.1)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()  # Functional API example


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_5 (Embedding)         │ (None, 20, 32)         │       722,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 746,945 (2.85 MB)

 Trainable params: 746,945 (2.85 MB)

 Non-trainable params: 0 (0.00 B)

In [140]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [142]:
model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=2)

Epoch 1/20
191/191 - 12s - 62ms/step - accuracy: 0.7054 - loss: 0.5532 - val_accuracy: 0.7886 - val_loss: 0.4635
Epoch 2/20
191/191 - 5s - 25ms/step - accuracy: 0.8941 - loss: 0.2815 - val_accuracy: 0.7833 - val_loss: 0.4810
Epoch 3/20
191/191 - 4s - 23ms/step - accuracy: 0.9512 - loss: 0.1430 - val_accuracy: 0.7820 - val_loss: 0.5927
Epoch 4/20
191/191 - 5s - 24ms/step - accuracy: 0.9739 - loss: 0.0867 - val_accuracy: 0.7551 - val_loss: 0.6573
Epoch 5/20
191/191 - 5s - 25ms/step - accuracy: 0.9814 - loss: 0.0623 - val_accuracy: 0.7630 - val_loss: 0.6409
Epoch 6/20
191/191 - 5s - 24ms/step - accuracy: 0.9877 - loss: 0.0439 - val_accuracy: 0.7531 - val_loss: 0.8460
Epoch 7/20
191/191 - 5s - 24ms/step - accuracy: 0.9874 - loss: 0.0408 - val_accuracy: 0.7472 - val_loss: 0.9474
Epoch 8/20
191/191 - 5s - 24ms/step - accuracy: 0.9915 - loss: 0.0334 - val_accuracy: 0.7525 - val_loss: 0.8654
Epoch 9/20
191/191 - 5s - 24ms/step - accuracy: 0.9913 - loss: 0.0290 - val_accuracy: 0.7525 - val_loss

In [143]:
predictions = model.predict(train_padded)
prediction = [1 if p>0.5 else 0 for p in predictions]

191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step


In [144]:
# Assuming train_sentences, train_labels, and predictions are defined
print("Train Sentences (10 to 19):")
print(train_sentences[10:20])

print("\nTrain Labels (10 to 19):")
print(train_labels[10:20])

print("\nPredictions (10 to 19):")
print(predictions[10:20])


Train Sentences (10 to 19):
['three people died heat wave far'
 'haha south tampa getting flooded hah wait second live south tampa gonna gonna fvck flooding'
 'raining flooding florida tampabay tampa 18 19 days ive lost count'
 'flood bago myanmar arrived bago'
 'damage school bus 80 multi car crash breaking' 'whats man' 'love fruits'
 'summer lovely' 'car fast' 'goooooooaaaaaal']

Train Labels (10 to 19):
[1 1 1 1 1 0 0 0 0 0]

Predictions (10 to 19):
[[9.9957430e-01]
 [9.9951118e-01]
 [9.9965203e-01]
 [9.9972808e-01]
 [9.9960971e-01]
 [3.5280973e-05]
 [4.0217277e-05]
 [5.1226565e-05]
 [1.3267476e-03]
 [7.2677183e-05]]
